In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    "E:\\data",  # Path to your dataset
    target_size=(224, 224),
    batch_size=8,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
   "E:\\data" ,  # Path to your dataset
    target_size=(224, 224),
    batch_size=8,
    class_mode='binary',
    subset='validation'
)

# Check class indices
print("Class indices:", train_generator.class_indices)

# Check the number of samples in each subset
print("Training samples:", train_generator.samples)
print("Validation samples:", validation_generator.samples)


Found 34 images belonging to 2 classes.
Found 7 images belonging to 2 classes.
Class indices: {'ai_generated': 0, 'original': 1}
Training samples: 34
Validation samples: 7


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

# Load base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Build model
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

# Evaluate model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy:.2f}')


Epoch 1/5
5/5 [==============================] - 7s 1s/step - loss: 1.5599 - accuracy: 0.4706 - val_loss: 1.1082 - val_accuracy: 0.4286
Epoch 2/5
5/5 [==============================] - 6s 1s/step - loss: 0.5981 - accuracy: 0.6765 - val_loss: 0.5060 - val_accuracy: 0.5714
Epoch 3/5
5/5 [==============================] - 6s 1s/step - loss: 0.2579 - accuracy: 0.9118 - val_loss: 0.4695 - val_accuracy: 0.8571
Epoch 4/5
5/5 [==============================] - 6s 1s/step - loss: 0.0742 - accuracy: 1.0000 - val_loss: 0.5890 - val_accuracy: 0.7143
Epoch 5/5
1/1 [==============================] - 1s 898ms/step - loss: 0.5822 - accuracy: 0.7143
Validation Accuracy: 0.71


In [3]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize to [0, 1]
    
    prediction = model.predict(img_array)
    return 'Original' if prediction < 0.5 else 'AI-generated'

# Test the model with some images
image_paths = [
    "E:\\data\\original\\image1.jpg",
    # Add paths to your images
]

for img_path in image_paths:
    result = predict_image(model, img_path)
    print(f'{img_path} is {result}')


1/1 [==============================] - 0s 284ms/step
E:\data\original\image1.jpg is AI-generated
